# gmxapi Python package overview

```
Author                : M. Eric Irrgang
Goal                  : Understand the gmxapi UI concepts
Time                  : 15 minutes
Prerequisites         : Familiarity with GROMACS MD simulation set up.
Software requirements : GROMACS, gmxapi
Tested for            : GROMACS2021
```

# General information

This notebook illustrates the Python interface for gmxapi.
Additional design aspects are illustrated where possible.

 - Prerequisites
     - GROMACS2021 with shared library support (standard)
     - gmxapi 0.2 Python package
 - Notebook Version
     - 2
 - References
     - [gmxapi manual](https://manual.gromacs.org/current/gmxapi/index.html)
     - [GROMACS CLI manual](https://manual.gromacs.org/current/user-guide/cmdline.html#commands-by-name)


# Python module dependencies
Import modules we will use in this notebook.

In [ ]:
# Import Python standard library tools.
import os
import shutil
from pathlib import Path

In [ ]:
# Import gmxapi package
import gmxapi as gmx

# Self-check

In [ ]:
gmx.version.api_is_at_least(0,2)

In [ ]:
# Make sure the *gmx* command line interface is accessible.
cli = Path(shutil.which('gmx'))

assert cli.exists()
cli

In [ ]:
# Work around resource management limitations
from gmxapi.simulation.mdrun import ResourceManager as _ResourceManager
_ResourceManager.mdrun_kwargs = {'threads': 1}

# Check working directories

In [ ]:
# What is our current working directory?
os.getcwd()

In [ ]:
# Identify the working directory that we will try to use.
nb_root = Path('usage').absolute()
# Confirm the availability of inputs.
assert (nb_root/'em.tpr').exists()

In [ ]:
os.listdir(nb_root)

# Manipulate simulation input.

## Acquire a handle to a TPR run input file.

In [ ]:
# Consider the *read_tpr* command.
gmx.read_tpr()

In [ ]:
help(gmx.read_tpr)

We will borrow a simulation input file from another tutorial.
*em.tpr* is generated with the [pipeline-basics.ipynb](pipeline-basics.ipynb) notebook.

## Warning
The GROMACS library emits uncatchable fatal errors that will kill the Jupyter notebook kernel.
Please file issues at https://gitlab.com/gromacs/gromacs/-/issues when you encounter such problems.

Regarding the following case, the bug in `read_tpr` has been noted.

In [ ]:
# Check for file existence before providing to read_tpr.
assert os.path.exists('usage/em.tpr')
em = gmx.read_tpr('usage/em.tpr')

## Explore the object model

In [ ]:
em

In [ ]:
# Inspect *em* and its member hierarchy with tab completion, dir(), and help()
# Take a look at em.node_uid
em.

In [ ]:
help(em.run)

In [ ]:
# Appologies: This isn't as navigable as we would like
help(em.output)

In [ ]:
# Take a look at em.output.parameters
em.output.

In [ ]:
help(em.output.parameters)

In [ ]:
em.output.parameters.description

In [ ]:
help(em.output.parameters.resource_manager)

In [ ]:
em.output.parameters.result()

In [ ]:
# Poke around at the hidden members by starting to name an attribute with `_` and then tab-completing
em.output._

In [ ]:
em.output._simulation_input.result()

## Use the modify_input command.

In [ ]:
modified = gmx.modify_input(em, parameters={'ld-seed': 0})

In [ ]:
# What is this thing?
modified

The modify_input command does not provide much directly to the user in gmxapi 0.2.
It is mostly an annotation, producing a reference to a conceptual point in the data flow.

In [ ]:
modified.output.parameters.result()

In [ ]:
modified.output._simulation_input.result()

The *gmxapi.simulation* modules override several of the more basic gmxapi components.

In [ ]:
modified.output._simulation_input.resource_manager

## Use the modified simulation input.

In [ ]:
sim = gmx.mdrun(modified)

In [ ]:
sim

In [ ]:
sim.output.trajectory.result()

# Unstable interfaces: lower level API bindings.

In [ ]:
from gmxapi.simulation import fileio
help(fileio)

In [ ]:
em = fileio.read_tpr('usage/em.tpr')
em

In [ ]:
em.parameters.extract()

In [ ]:
em.parameters.set('ld-seed', 0)

In [ ]:
em.parameters.extract()

In [ ]:
fileio.write_tpr_file('usage/edited.tpr', input=em)

In [ ]:
gmx.read_tpr('usage/edited.tpr').output.parameters.result()

# Exercise: Writing simple gmxapi commands
Use the `function_wrapper` decorator to copy a file from one directory to another.

In [ ]:
help(gmx.function_wrapper)

## Solution

In [ ]:
@gmx.function_wrapper(output={'path': str})
def cp(src: str, dst: str, output):
    infile = os.path.abspath(src)
    if not os.path.exists(infile):
        raise RuntimeError('Input file does not exist.')
    outfile = os.path.abspath(dst)
    if os.path.exists(outfile):
        if os.path.isfile(outfile):
            raise RuntimeError('Output file already exists.')
        else:
            if not os.path.isdir(outfile):
                raise RuntimeError('dst must be an existing directory or non-existing filename.')
            outfile = os.path.join(outfile, os.path.basename(infile))
    shutil.copy(infile, outfile)
    output.path = outfile

In [ ]:
cmd = cp(src='usage/edited.tpr', dst='/tmp')

In [ ]:
# Check out the dynamically generated output data proxy
cmd.output.

In [ ]:
assert os.path.exists(cmd.output.path.result())